In [1]:
import os, sys

import numpy as np
import pandas as pd 
from datasets import load_dataset

import importlib
from tqdm import tqdm
from joblib import Parallel, delayed
from copy import copy

from transformers import (
    AutoConfig,
    AutoTokenizer,
    FlaxAutoModelForSequenceClassification,
    HfArgumentParser,
    PretrainedConfig,
    TrainingArguments,
    is_tensorboard_available,
)

from flax.training.common_utils import get_metrics, onehot, shard


data_root = "/kaggle/input/feedback-prize-effectiveness/"
train = pd.read_csv("/kaggle/input/feedback-prize-effectiveness/train.csv")


/kaggle/working/jax/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
LABEL_MAPPING = {"Ineffective": 0, "Adequate": 1, "Effective": 2}

def _prepare_training_data_helper(args, tokenizer, df, is_train):
    training_samples = []
    for _, row in tqdm(df.iterrows(), total=len(df)):
        idx = row["essay_id"]
        discourse_text = row["discourse_text"]
        discourse_type = row["discourse_type"]

        if is_train:
            filename = os.path.join(args.input, "train", idx + ".txt")
        else:
            filename = os.path.join(args.input, "test", idx + ".txt")

        with open(filename, "r") as f:
            text = f.read()

        encoded_text = tokenizer.encode_plus(
            discourse_type + " " + discourse_text,
            text,
            add_special_tokens=False,
        )
        input_ids = encoded_text["input_ids"]

        sample = {
            "discourse_id": row["discourse_id"],
            "input_ids": input_ids,
            # "discourse_text": discourse_text,
            # "essay_text": text,
            # "mask": encoded_text["attention_mask"],
        }

        if "token_type_ids" in encoded_text:
            sample["token_type_ids"] = encoded_text["token_type_ids"]

        label = row["discourse_effectiveness"]

        sample["label"] = LABEL_MAPPING[label]

        training_samples.append(sample)
    return training_samples


def prepare_training_data(df, tokenizer, args, num_jobs, is_train):
    training_samples = []

    df_splits = np.array_split(df, num_jobs)

    results = Parallel(n_jobs=num_jobs, backend="multiprocessing")(
        delayed(_prepare_training_data_helper)(args, tokenizer, df, is_train) for df in df_splits
    )
    for result in results:
        training_samples.extend(result)

    return training_samples

In [3]:
sys.path.append("../configs")
cfg = copy(importlib.import_module("default_config").cfg)

# Load pretrained model and tokenizer
config = AutoConfig.from_pretrained(
    cfg.model_name_or_path,
    num_labels=cfg.num_labels,
    #finetuning_task=data_args.task_name,
    #use_auth_token=True if cfg.use_auth_token else None,
)
tokenizer = AutoTokenizer.from_pretrained(
    cfg.model_name_or_path,
    use_fast=not cfg.use_slow_tokenizer,
    #use_auth_token=True if cfg.use_auth_token else None,
)
model = FlaxAutoModelForSequenceClassification.from_pretrained(
    cfg.model_name_or_path,
    config=config,
    #use_auth_token=True if cfg.use_auth_token else None,
)

normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.
Some weights of the model checkpoint at google/bigbird-roberta-base were not used when initializing FlaxBigBirdForSequenceClassification: {('cls', 'seq_relationship', 'bias'), ('cls', 'predictions', 'transform', 'dense', 'kernel'), ('cls', 'predictions', 'transform', 'dense', 'bias'), ('cls', 'predictions', 'bias'), ('cls', 'predictions', 'transform', 'LayerNorm', 'bias'), ('cls', 'predictions', 'transform', 'LayerNorm', 'scale'), ('cls', 'seq_relationship', 'kernel')}
- This IS expected if you are initializing FlaxBigBirdForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing FlaxBigBirdForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequen

In [5]:
import json
from sklearn.model_selection import StratifiedKFold

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
## stratified Kfold for train dataframe using discourse_type and discourse_effectiveness
for fold, (train_index, valid_index) in enumerate(kf.split(train, train["discourse_type"], train["discourse_effectiveness"])):

    train_temp = train.iloc[train_index]
    valid_temp = train.iloc[valid_index]

    train_data = prepare_training_data(train_temp, tokenizer, cfg, num_jobs=96, is_train=True)
    val_data = prepare_training_data(valid_temp, tokenizer, cfg, num_jobs=96, is_train=True)

    df = pd.DataFrame.from_records(train_data)
    df.to_json(f"/kaggle/working/folds/train_{fold}.jsonl", orient="records", lines=True)

    df = pd.DataFrame.from_records(val_data)
    df.to_json(f"/kaggle/working/folds/valid_{fold}.jsonl", orient="records", lines=True)

    print("Fold:", fold)
    print("Train:", train_index)
    print("Valid:", valid_index)
    print("\n")

100%|██████████| 306/306 [00:00<00:00, 456.85it/s]
https://symbolize.stripped_domain/r/?trace=7f79a32213f4,7f79a32750bf,7f&map= 
*** SIGTERM received by PID 210570 (TID 210570) on cpu 35 from PID 207481; stack trace: ***
PC: @     0x7f79a32213f4  (unknown)  do_futex_wait.constprop.0
    @     0x7f78c36e6353        992  (unknown)
    @     0x7f79a32750c0  (unknown)  (unknown)
    @               0x80  (unknown)  (unknown)
https://symbolize.stripped_domain/r/?trace=7f79a32213f4,7f78c36e6352,7f79a32750bf,7f&map=a7dce6a9f5f70e08fccb1cadb4ea57b5:7f78af260000-7f78c3a602f0 
E0607 18:47:05.160205  210570 coredump_hook.cc:320] RAW: Remote crash gathering disabled for SIGTERM.
E0607 18:47:05.305296  210570 process_state.cc:774] RAW: Raising signal 15 with default behavior
100%|██████████| 76/76 [00:00<00:00, 499.28it/s]
Token indices sequence length is longer than the specified maximum sequence length for this model (9275 > 4096). Running this sequence through the model will result in indexing e

Fold: 0
Train: [    0     1     4 ... 36762 36763 36764]
Valid: [    2     3     7 ... 36733 36736 36746]




100%|██████████| 306/306 [00:00<00:00, 456.20it/s]
https://symbolize.stripped_domain/r/?trace=7f79a32213f4,7f79a32750bf,7f&map= 
*** SIGTERM received by PID 231759 (TID 231759) on cpu 31 from PID 207481; stack trace: ***
PC: @     0x7f79a32213f4  (unknown)  do_futex_wait.constprop.0
    @     0x7f78c36e6353        992  (unknown)
    @     0x7f79a32750c0  (unknown)  (unknown)
    @               0x80  (unknown)  (unknown)
https://symbolize.stripped_domain/r/?trace=7f79a32213f4,7f78c36e6352,7f79a32750bf,7f&map=a7dce6a9f5f70e08fccb1cadb4ea57b5:7f78af260000-7f78c3a602f0 
E0607 18:47:44.727387  231759 coredump_hook.cc:320] RAW: Remote crash gathering disabled for SIGTERM.
https://symbolize.stripped_domain/r/?trace=7f78c3742483,7f79a32750bf,7f78c35f4e06,7f78c36fbfea,7f78c3701a4c,7f78c37006d3,7f78c3700061,7f78c3a594ad,7f78c36e6dea,7f79a32750bf,7f&map=a7dce6a9f5f70e08fccb1cadb4ea57b5:7f78af260000-7f78c3a602f0 
E0607 18:47:44.729123  231759 process_state.cc:1067] RAW: Signal 11 raised at PC: 0x

Fold: 1
Train: [    0     2     3 ... 36759 36763 36764]
Valid: [    1     4     8 ... 36760 36761 36762]




100%|██████████| 306/306 [00:00<00:00, 615.38it/s]
Token indices sequence length is longer than the specified maximum sequence length for this model (9251 > 4096). Running this sequence through the model will result in indexing errors
100%|██████████| 306/306 [00:00<00:00, 435.95it/s]
https://symbolize.stripped_domain/r/?trace=7f79a32213f4,7f79a32750bf,7f&map= 
*** SIGTERM received by PID 253409 (TID 253409) on cpu 48 from PID 207481; stack trace: ***
PC: @     0x7f79a32213f4  (unknown)  do_futex_wait.constprop.0
    @     0x7f78c36e6353        992  (unknown)
    @     0x7f79a32750c0  (unknown)  (unknown)
    @               0x80  (unknown)  (unknown)
https://symbolize.stripped_domain/r/?trace=7f79a32213f4,7f78c36e6352,7f79a32750bf,7f&map=a7dce6a9f5f70e08fccb1cadb4ea57b5:7f78af260000-7f78c3a602f0 
E0607 18:48:25.784047  253409 coredump_hook.cc:320] RAW: Remote crash gathering disabled for SIGTERM.
E0607 18:48:25.867218  253409 process_state.cc:774] RAW: Raising signal 15 with default b

Fold: 2
Train: [    0     1     2 ... 36761 36762 36763]
Valid: [    5     6     9 ... 36755 36759 36764]




100%|██████████| 306/306 [00:00<00:00, 436.41it/s]
https://symbolize.stripped_domain/r/?trace=7f79a32213f4,7f79a32750bf,7f&map= 
*** SIGTERM received by PID 274535 (TID 274535) on cpu 29 from PID 207481; stack trace: ***
PC: @     0x7f79a32213f4  (unknown)  do_futex_wait.constprop.0
    @     0x7f78c36e6353        992  (unknown)
    @     0x7f79a32750c0  (unknown)  (unknown)
    @               0x80  (unknown)  (unknown)
https://symbolize.stripped_domain/r/?trace=7f79a32213f4,7f78c36e6352,7f79a32750bf,7f&map=a7dce6a9f5f70e08fccb1cadb4ea57b5:7f78af260000-7f78c3a602f0 
E0607 18:49:06.100985  274535 coredump_hook.cc:320] RAW: Remote crash gathering disabled for SIGTERM.
https://symbolize.stripped_domain/r/?trace=7f78c3742483,7f79a32750bf,7f78c35f4e06,7f78c36fbfea,7f78c3701a4c,7f78c37006d3,7f78c3700061,7f78c3a594ad,7f78c36e6dea,7f79a32750bf,7f&map=a7dce6a9f5f70e08fccb1cadb4ea57b5:7f78af260000-7f78c3a602f0 
E0607 18:49:06.104647  274535 process_state.cc:1067] RAW: Signal 11 raised at PC: 0x

Fold: 3
Train: [    0     1     2 ... 36761 36762 36764]
Valid: [   10    22    23 ... 36745 36756 36763]




100%|██████████| 306/306 [00:00<00:00, 430.38it/s]
https://symbolize.stripped_domain/r/?trace=7f79a32213f4,7f79a32750bf,7f&map= 
*** SIGTERM received by PID 295672 (TID 295672) on cpu 13 from PID 207481; stack trace: ***
PC: @     0x7f79a32213f4  (unknown)  do_futex_wait.constprop.0
    @     0x7f78c36e6353        992  (unknown)
    @     0x7f79a32750c0  (unknown)  (unknown)
    @               0x80  (unknown)  (unknown)
https://symbolize.stripped_domain/r/?trace=7f79a32213f4,7f78c36e6352,7f79a32750bf,7f&map=a7dce6a9f5f70e08fccb1cadb4ea57b5:7f78af260000-7f78c3a602f0 
E0607 18:49:47.035730  295672 coredump_hook.cc:320] RAW: Remote crash gathering disabled for SIGTERM.
E0607 18:49:47.144310  295672 process_state.cc:774] RAW: Raising signal 15 with default behavior
100%|██████████| 76/76 [00:00<00:00, 482.96it/s]
Token indices sequence length is longer than the specified maximum sequence length for this model (9251 > 4096). Running this sequence through the model will result in indexing e

Fold: 4
Train: [    1     2     3 ... 36762 36763 36764]
Valid: [    0    13    14 ... 36748 36753 36757]




In [44]:
# import json
# from sklearn.model_selection import StratifiedKFold

# kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
# ## stratified Kfold for train dataframe using discourse_type and discourse_effectiveness
# for fold, (train_index, valid_index) in enumerate(kf.split(train, train["discourse_type"], train["discourse_effectiveness"])):

#     train_temp = train.iloc[train_index]
#     valid_temp = train.iloc[valid_index]

#     train_data = prepare_training_data(train_temp, tokenizer, cfg, num_jobs=32, is_train=True)
#     val_data = prepare_training_data(valid_temp, tokenizer, cfg, num_jobs=32, is_train=True)

#     df = pd.DataFrame.from_records(train_data)
#     df.to_json(f"/kaggle/working/folds/train_{fold}.jsonl", orient="records", lines=True)

#     df = pd.DataFrame.from_records(val_data)
#     df.to_json(f"/kaggle/working/folds/valid_{fold}.jsonl", orient="records", lines=True)

#     print("Fold:", fold)
#     print("Train:", train_index)
#     print("Valid:", valid_index)
#     print("\n")

100%|██████████| 229/229 [00:00<00:00, 415.79it/s]
https://symbolize.stripped_domain/r/?trace=7f26ffdd18fd,7f26ffd7f0bf&map= 
*** SIGTERM received by PID 201426 (TID 201426) on cpu 51 from PID 88996; stack trace: ***
PC: @     0x7f26ffdd18fd  (unknown)  (unknown)
    @     0x7f261dfb7353        992  (unknown)
    @     0x7f26ffd7f0c0  (unknown)  (unknown)
    @ ... and at least 1 more frames
https://symbolize.stripped_domain/r/?trace=7f26ffdd18fd,7f261dfb7352,7f26ffd7f0bf&map=a7dce6a9f5f70e08fccb1cadb4ea57b5:7f2609b31000-7f261e3312f0 
E0607 18:44:49.296827  201426 coredump_hook.cc:320] RAW: Remote crash gathering disabled for SIGTERM.


Fold: 0
Train: [    0     1     4 ... 36762 36763 36764]
Valid: [    2     3     7 ... 36733 36736 36746]




 10%|█         | 94/919 [00:00<00:01, 483.61it/s]]

In [7]:
## data collator with dynamic padding
# def train_data_collator(rng:)
import jax
import datasets
from typing import Any, Callable, Dict, Optional, Tuple

rng = jax.random.PRNGKey(1)#cfg.seed)
dropout_rngs = jax.random.split(rng, jax.local_device_count())

Array = Any
Dataset = datasets.arrow_dataset.Dataset
PRNGKey = Any

In [50]:
def train_data_collator(rng: PRNGKey, dataset: Dataset, batch_size: int):
    """Returns shuffled batches of size `batch_size` from truncated `train dataset`, sharded over all local devices."""
    steps_per_epoch = len(dataset) // batch_size
    perms = jax.random.permutation(rng, len(dataset))
    perms = perms[: steps_per_epoch * batch_size]  # Skip incomplete batch.
    perms = perms.reshape((steps_per_epoch, batch_size))

    for perm in perms:
        batch = dataset[perm]
        discourse_id, input_ids, labels = dataset[perm]['discourse_id'], dataset[perm]['input_ids'], dataset[perm]['label']
        batch.pop("discourse_id", None)
        batch = {"input_ids": np.array(input_ids), "mask": [np.ones_like(x) for x in input_ids], "label": np.array(labels)}

        # calculate max token length of this batch
        batch_max = max([len(ids) for ids in batch["input_ids"]])
        # add padding
        if tokenizer.padding_side == "right":
            batch["input_ids"] = [s + (batch_max - len(s)) * [tokenizer.pad_token_id] for s in batch["input_ids"]]
        else:
            batch["input_ids"] = [(batch_max - len(s)) * [tokenizer.pad_token_id] + s for s in batch["input_ids"]]

        batch['input_ids'] = np.stack(batch['input_ids'])
        
        masks = np.zeros_like(batch['input_ids'])
        masks[batch['input_ids'] != tokenizer.pad_token_id] = 1
        batch['mask'] = masks

        batch = {k: np.array(v) for k, v in batch.items()}
        batch = shard(batch)
        yield batch

In [51]:
train_dataset = load_dataset("json", data_files="/kaggle/working/folds/valid_0.jsonl", split="train")
train_loader = train_data_collator(rng, train_dataset, cfg.per_device_train_batch_size)

In [52]:
for batch in train_loader:
    print(batch)
    break

938
(4, 938)
{'input_ids': array([[  898,  4461, 39341, ...,     0,     0,     0],
       [21360,  1507,   712, ...,     0,     0,     0],
       [20217,  4055,  8090, ...,   100,   321,   100],
       [23259,   415,   993, ...,     0,     0,     0]]), 'mask': array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 1, 1, 1],
       [1, 1, 1, ..., 0, 0, 0]]), 'label': array([0, 1, 1, 0])}


/tmp/ipykernel_207481/2323644318.py:12: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  batch = {"input_ids": np.array(input_ids), "mask": [np.ones_like(x) for x in input_ids], "label": np.array(labels)}


In [56]:
batch['mask'][1], batch['input_ids'][1]

(array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [57]:
import jax.profiler

In [58]:
server = jax.profiler.start_server(9999)


In [59]:
server

: 